# Automate Boring Stuff with Python
## Section 15: Emails

Usecase: send email notifications if a programs finds sth in, eg, a spreadsheet.

Email protocols are as old as the internet and not user friendly. That's why we get many old-fashioned byte literals `b''` and weird output.

### Send emails: SMTP

In [ ]:
import smtplib

In [ ]:
# Create connection to an SMTP server with port 587 (that's the usual port)
conn = smtplib.SMTP('smtp.gmail.com', 587)

In [ ]:
# Start connection (helo -> ehlo); 250 answer means connection is OK
conn.ehlo()

In [ ]:
# Start TLS encryption (for securely sending our PW); 220 answer means everything OK
conn.starttls()

In [ ]:
# In order to user the Google mail, we need to create an App Password or a Google specific password
# https://support.google.com/accounts/answer/185833?hl=en
# Steps
# - go to https://myaccount.google.com/
# - select Security
# - activate 2-Step Verification
# - in Signing in to Google, select App passwords
# - create a new passoword which will be used by the script (these passwords are tied to specific devices and programs)

In [ ]:
# Log in with usermail + pw
# WARNING: DO NOT UPLOAD FILE WITH PW ANYWHERE!
usermail = 'username@gmail.com'
pw = '...' # DO NOT SHARE! Insert here the app-specific pw we generated
conn.login(usermail,pw)

In [ ]:
# The structure of an email fix and it should be "Subject: <My Subject>\n\n<Body text>"
EmailText = '''Subject: My nice Subject\n\n
Dear Mikel,\n
This is a test.\n
All the best,\n
Mikel
'''

In [ ]:
EmailText

In [ ]:
# Send email; return is a dictionary with emails not sent, so an empty dict is a good signal
# from email first
# to emails second - we can write to several, see docs; but companies cap number of emails to 150 usually
conn.sendmail(usermail, 'mxagar@gmail.com', EmailText)

In [ ]:
conn.quit()

### Receive/read emails: IMAP

Email protocols are very old and not user friendly.
Python has the library `imaplib`, but we won't use it.
Instead, we're going to use `imapclient` and `pyzmail`.

In [ ]:
# Install modules
# pip/3/conda install imapclient/pyzmail

In [4]:
import imapclient

In [5]:
# Connect to the IMAP server of the email provider
conn = imapclient.IMAPClient('imap.gmail.com', ssl=True)

In [ ]:
conn.login('username@gmail.com', 'app-specific-pw')

In [ ]:
# List all folders we have
conn.list_folders()

In [ ]:
# Usually, we connect to the INBOX folder
conn.select_folder('INBOX', readonly=True)

In [ ]:
# Get the IDs of the emails since a spacific date
# We select an ID from the list to get the email content
# The search function has many keys - see the docs: BEFORE, SUBJECT/BODY/TEXT string, SEEN/UNSEEN, etc.
UIDs = conn.search(['SINCE 20-Aug-2020'])

In [ ]:
# Get the raw message of email with ID 47474
# Although the content is cryptic, we can already understand what's inside of it
rawMessage = conn.fetch([47474], ['BODY[]', 'FLAGS'])

In [6]:
# Raw message needs to be parsed
# This is done wth pyzmail

In [7]:
import pyzmail

In [ ]:
# To parse the message we need to pass its ID again and specify we want the body
message = pyzmail.PyzMessage.factory(rawMessage[47474][b'BODY[]'])

In [ ]:
# Get email subject
message.get_subject()

In [ ]:
# Get adresses
message.get_adresses('from')

In [ ]:
message.get_adresses('to')

In [ ]:
message.get_adresses('bcc')

In [ ]:
# Do we have plain text or HTML email?
message.text_part == None

In [ ]:
message.html_part == None

In [ ]:
# Get plain text message (usually UTF-8 encoding is used)
message.text_part.get_payload().decode('UTF-8')

In [ ]:
# If we want to delete a message: open folder not in readonly mode
# conn.select_folder('INBOX', readonly=False)
# Select email ID and delete it
conn.delete_messages([47474])

In [ ]:
# Close session
conn.logout()